<h1 id="原生依赖管理">原生依赖管理</h1>
<blockquote>
<p>不推荐使用</p>
</blockquote>

In [ ]:
!pip freeze > dirty.requirements.txt

In [ ]:
%pip install -r dirty.requirements.txt -i https://mirrors.tencent.com/pypi/simple

<h1 id="适用于Binder的依赖管理">适用于<span style="font-size: 24pt;"><a style="font-weight: 400; color: #3598db;" href="https://mybinder.org/" target="_blank" rel="noopener">Binder</a></span>的依赖管理</h1>

<p><span style="color: #169179;">生成 binder 专用的依赖清单（linux）应当在 linux 环境下进行，Windows 电脑可使用 WSL2，建议的发行版是 Debian 11</span></p>
<p><span style="color: #169179;">首先运行命令 <code>pip -V</code> 检查，Python 版本应当与 <code>runtime.txt</code> 约定的一致</span></p>
<p><span style="color: #169179;">确认生成 requirements.txt &nbsp;</span><span style="color: #169179;"><code>pip-compile --quiet --upgrade --resolver=backtracking requirements.in</code></span></p>
<p>&nbsp;</p>

<h1>甜蜜蜜的依赖管理</h1>

<p><span style="color: #169179;">这是绝佳的依赖管理，只需要保持一致的 Windows + Python 3.10</span></p>
<p><span style="color: #169179;">根据 README.md 复苏说明，随时随地部署 Jupyter Lab ，就如同安装普通软件一样简单</span></p>

硬盘I/O性能会显著影响 jupyter 运行性能！
假如 conda 安装位置所在硬盘I/O性能较差，可选的步骤是指定新虚拟环境的路径 `conda create -p G:\\BDC\\conda\\lab python=3.10` ，其中 `lab` 是虚拟环境名称，通过指定路径方式创建的虚拟环境与普通虚拟环境不同，使用 `conda activate G:\\BDC\\conda\\lab` 激活而不是 `conda activate lab`

使用 vscode 重新打开本 notebook 文件，切换虚拟环境为上面创建的虚拟环境，运行下面的命令

如果 vscode 提示需要 ipykernel 包，点击【安装】选项，会自动安装。如果是指定路径创建的虚拟环境，ipykernel 包可能无法由 vscode 成功安装，解决办法：在终端环境激活上面创建的虚拟环境，使用 `pip install ipykernel` 安装 ipykernel 包，然后执行 `python -m ipykernel install --user --name=lab` 注册内核即可（特别说明：--name=lab 中的 lab 并非必须和虚拟环境名称一致）

In [ ]:
# 已安装则不需要重复运行此命令
%pip install pip-tools -i https://mirrors.tencent.com/pypi/simple

### 备份/恢复 二选一

### 生成依赖清单

In [12]:
# 生成 windows AMD64平台的依赖清单
# win.requirements.in 的 UTF-8 编码请不要更改，不要在里面写入中文
# 运行前，检查所有已安装包的可执行文件是否均未在运行（例如 jupyter），避免占用或锁定导致的报错。
# 使用的源会影响是否找到所有 win.requirements.in 中列出的包（主要是指定了版本范围的），这是由于有些镜像收录不全或者版本更新延后，此时如果网络允许可以使用官方 PyPi 源 -i https://pypi.org/simple
# 一般情况下推荐使用腾讯云镜像加速 -i https://mirrors.tencent.com/pypi/simple
# 特别注意：下面的命令只有在本 notebook 文件在 vscode 中打开并选择了正确的虚拟环境下才可以直接执行，其他情况需要作出适当修改
!python -m piptools compile --quiet --upgrade --resolver=backtracking win.requirements.in --output-file win.requirements.txt -i https://mirrors.tencent.com/pypi/simple

### 根据依赖清单安装包

> 可使用 `pip list` 检查已安装的包， `pip list --outdated` 检查过时的包

In [ ]:
!pip-sync win.requirements.txt --pip-args "--quiet --retries 10 --timeout 30 --index-url https://mirrors.tencent.com/pypi/simple"

### 启动 jupyter lab

在 PowerShell 终端运行下面的命令

```powershell
jupyter lab --port='6969' --ip='*' --no-browser --allow-root --notebook-dir='D:\\TEMP\\jlab\\notebook'
```

--notebook-dir 传参因人而异，不出意外的话复苏成功。在终端 CTRL + 单击链接或者手动打开浏览器访问 [http://127.0.0.1:6969/lab](http://127.0.0.1:6969/lab) ，时间交给你了

或者，直接运行下面的代码块 FH92Y，然后手动打开浏览器访问 [http://127.0.0.1:6969/lab](http://127.0.0.1:6969/lab)。

In [ ]:
# 代码块 FH92Y
!jupyter lab --port=6969 --ip=* --no-browser --allow-root --notebook-dir="D:\\TEMP\\jlab\\notebook"

并不推荐直接运行上面代码块 FH92Y 这种方式，仅供测试使用，且需要 `netstat -ano | findstr :6969` 配合 `taskkill /f /pid` 手动结束进程。

最后，可以通过下面的 C++ 代码构建一个可执行程序，只需要双击该程序即可快速运行 jupyter lab，也可以完成集成到其他软件，加入开机自启等操作

注意事项：

1. `conda activate lab` 可能需要根据前面的操作调整为 `conda activate G:\\BDC\\conda\\lab`
2. --notebook-dir 传参修改为正确的路径

```cpp
#include <stdlib.h>
#include <Windows.h>
#include <shellapi.h>
#include <WinUser.h>
#pragma comment( linker, "/subsystem:\"windows\" /entry:\"mainCRTStartup\"" ) // 设置入口地址，隐藏控制台窗口

using namespace std;

int main()
{
    // 如果 python 是 64位的，则生成对应的 x64 (64位) 程序
    ShellExecute(NULL, L"open", L"powershell", L"conda activate lab;jupyter lab --port='6969' --ip='*' --no-browser --allow-root --notebook-dir='D:\\TEMP\\jlab\\notebook'", NULL, SW_SHOWMINNOACTIVE);
    // 循环检测本地端口连通性，成功则自动打开指定链接。一般用不上，因为 6969 端口可能被占用，此时 jupyter lab 将使用 6970 端口，具体使用端口以控制台输出为准
    /*ShellExecute(NULL, L"open", L"powershell", L"do{$ff=Test-NetConnection 127.0.0.1 -p 6969;\
if($ff.TcpTestSucceeded -eq 'True'){start http://127.0.0.1:6969/lab}}until($ff.TcpTestSucceeded -eq 'False')", NULL, SW_HIDE);*/
}

/* 最后的那个参数，为窗口指定可视性方面的一个命令。 请用下述任何一个常数
* SW_HIDE 隐藏窗口，活动状态给令一个窗口
* SW_MINIMIZE 最小化窗口，活动状态给令一个窗口
* SW_RESTORE 用原来的大小和位置显示一个窗口，同时令其进入活动状态
* SW_SHOW 用当前的大小和位置显示一个窗口，同时令其进入活动状态
* SW_SHOWMAXIMIZED 最大化窗口，并将其激活
* SW_SHOWMINIMIZED 最小化窗口，并将其激活
* SW_SHOWMINNOACTIVE 最小化一个窗口，同时不改变活动窗口
* SW_SHOWNA 用当前的大小和位置显示一个窗口，不改变活动窗口
* SW_SHOWNOACTIVATE 用最近的大小和位置
* SW_SHOWNORMAL 正常显示
*/
```

<h1>初始化工作</h1>

In [ ]:
# REF https://nbdime.readthedocs.io/en/latest/
# 激活差异对比工具
# 对于 Binder 或者 docker ，这一步不需要手动执行
%nbdime config-git --enable --global

In [3]:
# 添加模板路径，jupyter_server_config.py文件路径和template_dirs路径因人而异
# 此实例中，实际模板存放位置为 D:\TEMP\jlab\notebook\repo\templates\jupyterlab_templates ，放在 jupyterlab_templates 子文件夹是规定
# 注意是追加模式，此代码块应当仅执行一次。可以用VSCode打开jupyter_server_config.py文件手动修改
with open(r"C:\Users\Administrator\.jupyter\jupyter_server_config.py", "a", encoding="utf-8") as f:
    f.write('''
c.JupyterLabTemplates.template_dirs = [r"D:\\TEMP\\jlab\\notebook\\repo\\templates"]
c.JupyterLabTemplates.include_default = True
c.JupyterLabTemplates.include_core_paths = True
''')